In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import implicit
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from numpy.core.umath_tests import inner1d

/home/raghavjajodia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  if __name__ == '__main__':


In [2]:
full_data = pd.read_csv('data/final_data.csv', index_col = 0)

/home/raghavjajodia/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
class cdfTransformer(object):
    
    transformerSeries = {}
    
    def __init__(self, useridcolname, itemidcolname, ratingcolname):
        self.userid_colname = useridcolname
        self.itemid_colname = itemidcolname
        self.rating_colname = ratingcolname
    
    def getLowerOrEqualIndex(self,playtimeList, playtime):
        if playtime < playtimeList[0]:
            return 0
        ans = 0
        low = 0
        high = len(playtimeList) - 1
        while(low<=high):
            mid = low + (high-low)//2
            if playtime > playtimeList[mid]:
                ans = mid
                low = mid + 1
            elif playtime == playtimeList[mid]:
                return mid
            else:
                high = mid - 1
        return ans      
    
    def getNearestCdf(self,appid, playtime):
        playtimeList = self.transformerSeries[appid].index
        bestpos = self.getLowerOrEqualIndex(playtimeList, playtime)
        return self.transformerSeries[appid].iloc[bestpos]
    
    def fitTransform(self,tupledata):
        grouped1 = tupledata.groupby([self.itemid_colname,self.rating_colname]).count()
        grouped2 = grouped1.groupby(level=[0]).cumsum()
        grouped3 = grouped2.groupby(level = [0]).max()
        withcdf = grouped2/grouped3
        self.transformerSeries = pd.Series(withcdf[self.userid_colname],index=withcdf.index)
        withcdf_df = withcdf.reset_index(level=[0,1])
        withcdf_df.rename(columns={self.userid_colname:'temp_rating'}, inplace=True)
        finaltuple = pd.merge(withcdf_df,tupledata, on=[self.itemid_colname,self.rating_colname],how='inner',suffixes=('_newdf',''))
        finaltuple.drop(self.rating_colname, inplace=True, axis = 1)
        finaltuple.rename(columns={'temp_rating':self.rating_colname}, inplace=True)
        return finaltuple

    def Transform(self,tupledata):
        ansdata = tupledata.groupby([self.itemid_colname,self.rating_colname]).count().reset_index()
        ansdata.drop(self.userid_colname, inplace = True, axis = 1)
        ansdata['rating_temp'] =  ansdata.apply(lambda x: self.getNearestCdf(x[self.itemid_colname],x[self.rating_colname]), axis = 1)
        ansdata = pd.merge(ansdata,tupledata, on=[self.itemid_colname,self.rating_colname],how='inner',suffixes=('_newdf',''))
        ansdata.drop(self.rating_colname, axis = 1, inplace = True)
        ansdata.rename(columns = {"rating_temp":self.rating_colname}, inplace = True)         
        return ansdata

In [4]:
def train_test_split(dff, split_ratio = 0.8):
    
    tr_sample = dff.groupby('sid').apply(lambda x:x.sample(frac = 0.8))
    tr_sample_index = list(zip(*tr_sample.index))[1]
    te_sample = dff[(dff.index.isin(tr_sample_index) == False)]
    
    tr_sample.index.rename(['id', 'appid_level'], inplace=True)
    tr_sample.reset_index(drop = True, inplace = True)
    te_sample.reset_index(drop = True, inplace = True)
    
    return tr_sample, te_sample

In [5]:
class matrixFactorizer(object):
    
    def __init__(self, latentFactors = 3, max_iterations = 20, reg = 0.01):        
        self.numLatent = latentFactors
        self.max_iterations = max_iterations
        self.reg = reg
    
    def fit(self,train_ratingsbyuser, train_ratingsbyitem, total_users, total_items):
        self.U = np.random.randn(total_users, self.numLatent) / self.numLatent
        self.V = np.random.randn(self.numLatent, total_items) / self.numLatent
        
        for t in range(self.max_iterations):
          # update U
          for i in range(total_users):
            if i in train_ratingsbyuser:
              matrix = np.zeros((self.numLatent, self.numLatent)) + self.reg*np.eye(self.numLatent)
              vector = np.zeros(self.numLatent)
              for j, r in train_ratingsbyuser[i]:
                matrix += np.outer(self.V[:,j], self.V[:,j])
                vector += (r)*self.V[:,j]
              self.U[i,:] = np.linalg.solve(matrix, vector)

          # update V
          for j in range(total_items):
            if j in train_ratingsbyitem:
              matrix = np.zeros((self.numLatent, self.numLatent)) + self.reg*np.eye(self.numLatent)
              vector = np.zeros(self.numLatent)
              for i, r in train_ratingsbyitem[j]:
                matrix += np.outer(self.U[i,:], self.U[i,:])
                vector += (r)*self.U[i,:]
              self.V[:,j] = np.linalg.solve(matrix, vector)
    
    def predict(self, user_u, item_i):
        return inner1d(self.U[user_u,:],np.transpose(self.V)[item_i,:])

In [6]:
def constructCodes(df) :
    """
    Maps 'steamid' to categorical code 'sid'
    Maps 'appid' to categorical code 'aid'
    Returns :
        1. Dataframe with columns 'sid', 'aid', 'playtime_forever'
        2. Dictionary mapping 'steamid' to 'sid'
        3. Dictionary mapping 'sid' to 'steamid'
        4. Dictionary mapping 'appid' to 'aid'
        5. Dictionary mapping 'aid' to 'appid'
    """
    coded_df = df.copy(deep = True)
    coded_df["steamid"] = coded_df["steamid"].astype("category")
    coded_df["appid"] = coded_df["appid"].astype("category")
    coded_df["sid"] = coded_df["steamid"].cat.codes
    coded_df["aid"] = coded_df["appid"].cat.codes
    
    sid_to_steamid = dict(enumerate(coded_df["steamid"].cat.categories))
    aid_to_appid = dict(enumerate(coded_df["appid"].cat.categories))
    steamid_to_sid = {v : k for k, v in sid_to_steamid.items()}
    appid_to_aid = {v : k for k, v in aid_to_appid.items()}
    
    coded_df.drop(["steamid", "appid"], axis = 1, inplace = True)
    
    return(coded_df, steamid_to_sid, sid_to_steamid, appid_to_aid, aid_to_appid)

In [7]:
def constructSparseMatrices(df) :
    """
    Constructs sparse matrices that will be used in ALS optimization
    Input : Dataframe with columns - 'sid', 'aid', 'playtime_forever'
    
    Returns :
        1. User x Item Sparse Matrix
        2. Item x User Sparse Matrix
    """
    data_useritem = sparse.csr_matrix((df["playtime_forever"], (df["sid"], df["aid"])))
    data_itemuser = sparse.csr_matrix((df["playtime_forever"], (df["aid"], df["sid"])))
    
    sid_unique = df["sid"].nunique()
    aid_unique = df["aid"].nunique()
    
    assert data_useritem.shape == (sid_unique, aid_unique)
    assert data_itemuser.shape == (aid_unique, sid_unique)
    
    return(data_useritem, data_itemuser)

In [8]:
def trainModel(data, factors, epochs, conf_func, alpha, lmbda) :
    """
    Builds and trains Implicit Matrix Factorization model.
    Input :
        1. data - Item x User Sparse Matrix
        2. factors - Number of latent factors
        3. epochs - Number of iterations of ALS over the training data
        4. conf_func - Confidence function
        5. alpha - Confidence parameter
        6. lmbda - Regularization parameter
        
    Output : model
    """
    model = implicit.als.AlternatingLeastSquares(factors = factors, regularization = lmbda, iterations = epochs)
    if conf_func == "linear" :
        model.fit(alpha * data)
    else :
        print("{} is not a valid choice for conf_func. Choose one of the following : 'linear'".format(conf_func))
        return(None)
    return(model)

In [9]:
def evaluateValidationLoss(item_factors, user_factors, testtuples, itemidcolname, useridcolname, ratingcolname):
    """
    Input :
        1. item_factors - items * latentsize
        2. user_factors - users * latentsize
    Output : validationloss
    """
    predictions = inner1d(item_factors[testtuples[itemidcolname],:],user_factors[testtuples[useridcolname],:])
    return np.sqrt((((testtuples[ratingcolname] - predictions)**2).dot(np.ones(testtuples.shape[0])))/testtuples.shape[0])

In [10]:
class MF():

    def __init__(self, K, alpha, beta, iterations, ratingsbyuser, ratingsbyitem, traintuples, ratingcolname, itemcolname, usercolname):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        - ratings by user
        - ratigns by item
        """

        self.num_users, self.num_items = len(ratingsbyuser.keys()), len(ratingsbyitem.keys()) 
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.ratingsbyuser = ratingsbyuser
        self.ratingsbyitem = ratingsbyitem
        self.traintuples = traintuples
        self.ratingcolname = ratingcolname
        self.itemcolname = itemcolname
        self.usercolname = usercolname

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.traintuples[self.ratingcolname])

        # Create a list of training samples
        #self.samples = [
        #    (i, j, self.R[i, j])
        #    for i in range(self.num_users)
        #    for j in range(self.num_items)
        #    if self.R[i, j] > 0
        #]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            self.traintuples.sample(frac=1).reset_index(drop=True)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        predictions_dot = inner1d(self.Q[self.traintuples[self.itemcolname],:],self.P[self.traintuples[self.usercolname],:])
        biases = self.b_i[self.traintuples[self.itemcolname]] + self.b_u[self.traintuples[self.usercolname]] + self.b
        predictions = predictions_dot + biases
        return np.sqrt((((self.traintuples[self.ratingcolname] - predictions)**2).dot(np.ones(self.traintuples.shape[0])))/self.traintuples.shape[0])        

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for index, row in self.traintuples.iterrows():
            i,j,r = int(row['sid']),int(row['aid']),row['playtime_forever']
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [11]:
#create coded data, and split in train,test
coded_data, steamidtosid, sidtosteamid, appidtoaid, aidtoappid = constructCodes(full_data)
train,test = train_test_split(coded_data)
transformer1 = cdfTransformer('sid','aid','playtime_forever')
traintuples = transformer1.fitTransform(train)
train.shape, test.shape, traintuples.shape

((2643226, 3), (660690, 3), (2643226, 3))

In [12]:
testtuples = transformer1.Transform(test)
testtuples.head()

,aid,playtime_forever,sid
0,0,0.097124,30
1,0,0.097124,73
2,0,0.097124,75
3,0,0.097124,76
4,0,0.097124,83


In [13]:
testtuples.shape, traintuples['aid'].nunique(), traintuples['sid'].nunique(), testtuples['aid'].nunique(), testtuples['sid'].nunique()

((660690, 3), 12982, 13845, 12858, 13845)

In [14]:
full_data['appid'].nunique()

12982

In [15]:
user_item_mat, item_user_mat = constructSparseMatrices(traintuples)

In [16]:
user_item_mat.shape

(13845, 12982)

In [18]:
factors = [5,10,50,100]
epochs = [10,50,100]
lambdas = [0.001,0.01,0.1,1]

allModels = {}
bestLoss = 10000000

for factor in factors:
    for epoch in epochs:
        for lamb in lambdas:
            tempmodel = trainModel(item_user_mat,factor, epoch,'linear',1,lamb)
            valLoss = evaluateValidationLoss(tempmodel.item_factors, tempmodel.user_factors, testtuples, 'aid', 'sid', 'playtime_forever')
            key = str(factor) + "_" + str(epoch) + "_" + str(lamb)
            allModels[key] = tempmodel
            
            if(valLoss < bestLoss):
                bestLoss = valLoss
                bestConfiguration = (factor, epoch, lamb)

100%|██████████| 10.0/10 [00:02<00:00,  4.34it/s]
100%|██████████| 10.0/10 [00:02<00:00,  4.10it/s]
100%|██████████| 10.0/10 [00:02<00:00,  3.86it/s]
100%|██████████| 10.0/10 [00:02<00:00,  3.66it/s]
100%|██████████| 50.0/50 [00:15<00:00,  3.02it/s]
100%|██████████| 50.0/50 [00:16<00:00,  3.09it/s]
100%|██████████| 50.0/50 [00:16<00:00,  3.06it/s]
100%|██████████| 50.0/50 [00:16<00:00,  3.05it/s]
100%|██████████| 100.0/100 [00:33<00:00,  3.04it/s]
100%|██████████| 100.0/100 [00:33<00:00,  2.94it/s]
100%|██████████| 100.0/100 [00:34<00:00,  2.93it/s]
100%|██████████| 100.0/100 [00:34<00:00,  2.93it/s]
100%|██████████| 10.0/10 [00:03<00:00,  2.70it/s]
100%|██████████| 10.0/10 [00:03<00:00,  2.74it/s]
100%|██████████| 10.0/10 [00:03<00:00,  2.83it/s]
100%|██████████| 10.0/10 [00:03<00:00,  2.58it/s]
100%|██████████| 50.0/50 [00:18<00:00,  2.76it/s]
100%|██████████| 50.0/50 [00:18<00:00,  2.84it/s]
100%|██████████| 50.0/50 [00:18<00:00,  2.74it/s]
100%|██████████| 50.0/50 [00:18<00:00,  2.

In [22]:
f = open('implicitresults2.txt','w')
for k in allModels:
    mod = allModels[k]
    valLoss = evaluateValidationLoss(mod.item_factors, mod.user_factors, testtuples, 'aid', 'sid', 'playtime_forever')
    f.write(str(k) + "   " + str(valLoss) + '\n')
f.close()
            

In [19]:
f = open('implicitresults.txt','a')
f.write(str(allModels))
f.cstrlose()

In [91]:
model = trainModel(item_user_mat, 10, 10, 'linear', 1, 0.1)

100%|██████████| 10.0/10 [00:02<00:00,  4.17it/s]


In [143]:
bestConfiguration

(50, 10, 0.1)

In [134]:
evaluateValidationLoss(model.item_factors, model.user_factors, testtuples, 'aid', 'sid','playtime_forever')

0.4449343065072283

In [21]:
temptraintuples = traintuples.copy(deep=True)
temptraintuples['tup'] = list(zip(temptraintuples['sid'], temptraintuples['playtime_forever']))
ratingsbyitem = temptraintuples.groupby('aid')['tup'].apply(list).to_dict()

temptraintuples2 = traintuples.copy(deep=True)
temptraintuples2['tup'] = list(zip(temptraintuples2['aid'], temptraintuples2['playtime_forever']))
ratingsbyuser = temptraintuples2.groupby('sid')['tup'].apply(list).to_dict()

In [29]:
matFact1 = matrixFactorizer(latentFactors=20)
matFact1.fit(ratingsbyuser, ratingsbyitem, traintuples['sid'].nunique(), traintuples['aid'].nunique())

In [30]:
evaluateValidationLoss(np.transpose(matFact1.V), matFact1.U, testtuples, 'aid', 'sid', 'playtime_forever')

0.286446025915094